In [2]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

import matlab.engine as engi
import matlab as mat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from pyearth import Earth

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt

# Data Load and other util function

In [9]:
def load_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    commit_guru_df = pd.read_csv(commit_guru_path)
    cols = understand_df.columns.tolist()
    
    commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
                                                   'author_email','fileschanged','author_date',
                                                   'author_date_unix_timestamp', 'commit_message',
                                                  'classification', 'fix', 'contains_bug','fixes',],axis=1)
    commit_cols = commit_guru_df.columns.values.tolist()
    df = understand_df.merge(commit_guru_df,on='commit_hash')
    cols = df.columns.tolist()
    cols = cols[1:] + [cols[0]]
    df = df[cols]
    for item in ['Kind','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    commit_cols.remove('commit_hash')
    commit_cols.append('Name')
    commit_cols.append('Bugs')
    df = df[commit_cols]
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

def get_spearmanr(df):
    df = df.drop('Bugs',axis = 1)
    file_corr = []
    for name in df.Name.unique():
        sub_df = df[df['Name'] == name]
        if sub_df.shape[0] < 2:
            continue
        sub_df = sub_df.drop('Name',axis = 1)
        rho, pval = stats.spearmanr(sub_df.values,sub_df.values,axis = 1)
        file_corr.append(np.median(rho[:,:int(len(rho)/2)]))
    return file_corr

# Getting List of projects

In [10]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

# Run the experiment

In [ ]:
project_corr = {}
for project in projects:
    try:
        if project == '.DS_Store':
            continue
#         if project != 'Pokemap':
#             continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        df = load_data(project)
        corr = get_spearmanr(df)
        project_corr[project] = corr
        print(np.median(corr))
    except Exception as e:
        print(e)
        continue

+++++++++++++++++   org.alloytools.alloy  +++++++++++++++++
1.0
+++++++++++++++++   qpython  +++++++++++++++++
0.8975244475938271
+++++++++++++++++   friendlychat-android  +++++++++++++++++
1.0
+++++++++++++++++   paho.mqtt.android  +++++++++++++++++
0.7546888753310097
+++++++++++++++++   paho.mqtt.java  +++++++++++++++++
0.5146208533181836
+++++++++++++++++   android-docs-samples  +++++++++++++++++
0.9545438358063261
+++++++++++++++++   Toasty  +++++++++++++++++
0.7712123747458672
+++++++++++++++++   android-mvvm-architecture  +++++++++++++++++
0.9452188332459757
+++++++++++++++++   recipes-rss  +++++++++++++++++
0.6049739778208676
+++++++++++++++++   HoloGraphLibrary  +++++++++++++++++
nan
+++++++++++++++++   HorizontalWheelView  +++++++++++++++++
0.8972417571861557
+++++++++++++++++   OpenMemories-Tweak  +++++++++++++++++
0.8848279522740883
+++++++++++++++++   HoldingButton  +++++++++++++++++
0.9800554747759365
+++++++++++++++++   ExpandableLayout  +++++++++++++++++
0.76200564971751

0.783568464961872
+++++++++++++++++   chips-input-layout  +++++++++++++++++
0.8989186161932762
+++++++++++++++++   elasticsearch-analysis-mmseg  +++++++++++++++++
0.8244134526185265
+++++++++++++++++   XUpdate  +++++++++++++++++
0.7363288705683797
+++++++++++++++++   jenkins-hipchat-plugin  +++++++++++++++++
0.725201477814461
+++++++++++++++++   spring-cloud-kubernetes  +++++++++++++++++
0.7672176308539945
+++++++++++++++++   cordova-imagePicker  +++++++++++++++++
0.5977961432506887
+++++++++++++++++   Android-SlideExpandableListView  +++++++++++++++++
0.6417759259656491
+++++++++++++++++   smarthome  +++++++++++++++++
0.665748386032949
+++++++++++++++++   richeditor-android  +++++++++++++++++
0.4559909814706388
+++++++++++++++++   CtCI-6th-Edition  +++++++++++++++++
0.43235746374428746
+++++++++++++++++   spring-mvc-showcase  +++++++++++++++++
0.7630853994490358
+++++++++++++++++   RMSwitch  +++++++++++++++++
0.8397918225669414
+++++++++++++++++   easy-adapter  +++++++++++++++++
0.657

0.6328194472548616
+++++++++++++++++   gs-uploading-files  +++++++++++++++++
0.6724878723728207
+++++++++++++++++   android-flowlayout  +++++++++++++++++
0.8446534870343866
+++++++++++++++++   dbeaver  +++++++++++++++++
0.6860468343787431
+++++++++++++++++   android-mvp-architecture  +++++++++++++++++
0.7565354962631877
+++++++++++++++++   otter  +++++++++++++++++
0.660655764575696
+++++++++++++++++   AlphabetIndex-Fast-Scroll-RecyclerView  +++++++++++++++++
0.4766463429249534
+++++++++++++++++   CalendarFX  +++++++++++++++++
0.7103753084008629
+++++++++++++++++   knife  +++++++++++++++++
0.7883773030488008
+++++++++++++++++   StepView  +++++++++++++++++
0.8626373626373626
+++++++++++++++++   Space-Navigation-View  +++++++++++++++++
0.7736651215900227
+++++++++++++++++   mqtt-client  +++++++++++++++++
0.6574685979306976
+++++++++++++++++   emoji-java  +++++++++++++++++
0.5861566745736971
+++++++++++++++++   reflections  +++++++++++++++++
0.5805230697645591
+++++++++++++++++   databus  

0.4051679226772044
+++++++++++++++++   gpmall  +++++++++++++++++
0.6208791208791209
+++++++++++++++++   openhab1-addons  +++++++++++++++++
0.7050812154541788
+++++++++++++++++   re2j  +++++++++++++++++
0.4311565686263878
+++++++++++++++++   mortar  +++++++++++++++++
0.728282097091017
+++++++++++++++++   vertx-sql-client  +++++++++++++++++
0.8402235739254607
+++++++++++++++++   groupie  +++++++++++++++++
0.5288969069308119
+++++++++++++++++   UrlImageViewHelper  +++++++++++++++++
0.5968533761211093
+++++++++++++++++   jgit-cookbook  +++++++++++++++++
0.6140427271571167
+++++++++++++++++   ice  +++++++++++++++++
0.5337415577833999
+++++++++++++++++   dashboard-demo  +++++++++++++++++
0.7525763472182971
+++++++++++++++++   qmq  +++++++++++++++++
0.582369382991664
+++++++++++++++++   Paper  +++++++++++++++++
0.6543290995068789
+++++++++++++++++   OpenNoteScanner  +++++++++++++++++
0.7828057320025196
+++++++++++++++++   plantuml-server  +++++++++++++++++
0.7589531680440772
+++++++++++++++++

0.6345310787392893
+++++++++++++++++   kumo  +++++++++++++++++
0.6861216994906438
+++++++++++++++++   hn-android  +++++++++++++++++
0.5943504826483824
+++++++++++++++++   coffee-gb  +++++++++++++++++
0.8482646724865005
+++++++++++++++++   postgresql-embedded  +++++++++++++++++
0.6435967483322658
+++++++++++++++++   spring-loaded  +++++++++++++++++


In [87]:
df = df.drop('Bugs',axis = 1)
for name in df.Name.unique():
    sub_df = df[df['Name'] == name]
    sub_df = sub_df.drop('Name',axis = 1)
    rho, pval = stats.spearmanr(sub_df.values,sub_df.values,axis = 1)
    print(np.median(rho))

1.0
1.0
0.982977093922288
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
1.0
1.0
0.9999999999999999
0.9999999999999999
1.0
1.0
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999

In [51]:
sub_df.values.shape

(7, 45)

In [43]:
x2n=np.random.randn(100,60)
y2n=np.random.randn(100,60)

In [82]:
rho, pval = stats.spearmanr(sub_df.values,sub_df.values,axis = 1)

In [109]:
rho[:,:int(len(rho)/2)]

array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]])